In [1]:
import pandas as pd
import numpy as np
import MySQLdb
import datetime

In [159]:
db = MySQLdb.connect("47.100.14.137","hyps44413","Nanhua")
db.set_character_set('gbk')

In [154]:
db = MySQLdb.connect("localhost","root","123456")
db.set_character_set('gbk')

In [ ]:
db.close()

In [ ]:
contract_list={'pp':'pp','jm':'jm','jd':'jd','fb':'fb','cs':'cs','bb':'bb','y':'y','v':'v','p':'p'
               ,'m':'m','l':'l','j':'j','c':'c','a':'a_1','b':'a_2','i':'i'}

# contract_id=['pp','jm','jd','fb','cs','bb','y','v','p','m','l','j','c','a','b','i']
contract_id = ['m','c']

for x in range(len(contract_id)):
    
    company_order = pd.read_excel('../期貨公司名稱/大商所/' + contract_id[x] + '_company_name.xlsx',encoding='gbk',header=None,index_col=None)
    databaseName1 = contract_list[contract_id[x]] + '_oi_data'
    databaseName2 = contract_list[contract_id[x]] + '_oi_original_data'
    
    code = contract_list[contract_id[x]]
    
    V_info_create(databaseName1, company_order)
    B_info_create(databaseName1, company_order)
    S_info_create(databaseName1, company_order)

    cursor = db.cursor()

    # sql = "CREATE DATABASE IF NOT EXISTS " + databaseName1
    # cursor.execute(sql)

    # sql = "CREATE DATABASE IF NOT EXISTS " + databaseName2
    # cursor.execute(sql)

#     create_futures_member(databaseName1)
#     create_non_futures_member(databaseName1)
#     create_total_volume(databaseName1)

    now_date = datetime.datetime(2002,3,15)
    end_date = datetime.datetime(2018,1,19)
    
    print(code)

    while now_date < end_date:
        split_date = now_date.strftime('%Y-%m-%d').split('-')
        table_date = split_date[0]+split_date[1]+split_date[2]
        
        try:
            data = pd.read_csv('../各交易所期貨持倉量/大商所/'+contract_id[x]+'/'+table_date+'.csv',encoding='gbk',engine='python')
            for i in range(3,len(data)-1):
                for j in range(12):
                    if(data.loc[i][j] == "-"):
                        data.loc[i][j] = None  #塞原始資料
#                         data.loc[i][j] = "NULL"  #塞分類資料
                    elif( (j==2 or j==3 or j==6 or j==7 or j==10 or j==11) and data.loc[i][j] != None):
                        data.loc[i][j] = data.loc[i][j].replace(',','')

#             total_insert(data,table_date,databaseName2,code)
#             futures_member_insert(data,table_date,databaseName1)
#             non_futures_member_insert(data,table_date,databaseName1)
#             total_volume_insert(data,table_date,databaseName1)

            V_info_insert(data, table_date, databaseName1)
            B_info_insert(data, table_date, databaseName1)
            S_info_insert(data, table_date, databaseName1)
            
            now_date = now_date + datetime.timedelta(days = 1)
            
        except Exception as e:
            now_date = now_date + datetime.timedelta(days = 1)
            print (e)


m
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20020315.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20020316.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20020317.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20020318.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20020319.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20020320.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20020321.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20020322.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20020323.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20020324.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20020325.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20020326.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20020327.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20020328.csv'
[Err

[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20020721.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20020722.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20020723.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20020724.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20020725.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20020726.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20020727.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20020728.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20020729.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20020730.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20020731.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20020801.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20020802.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20020803.csv'
[Errno

[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20021118.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20021119.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20021120.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20021121.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20021122.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20021123.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20021124.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20021125.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20021126.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20021127.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20021128.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20021129.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20021130.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20021201.csv'
[Errno

[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20030315.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20030316.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20030317.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20030318.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20030319.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20030320.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20030321.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20030322.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20030323.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20030324.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20030325.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20030326.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20030327.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20030328.csv'
[Errno

[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20030722.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20030723.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20030724.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20030725.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20030726.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20030727.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20030728.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20030729.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20030730.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20030731.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20030801.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20030802.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20030803.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20030804.csv'
[Errno

...

[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20050618.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20050619.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20050625.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20050626.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20050702.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20050703.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20050709.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20050710.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20050716.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20050717.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20050723.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20050724.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20050730.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20050731.csv'
[Errno

[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20060603.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20060604.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20060610.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20060611.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20060617.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20060618.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20060624.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20060625.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20060701.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20060702.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20060708.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20060709.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20060715.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20060716.csv'
[Errno

[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20070512.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20070513.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20070519.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20070520.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20070526.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20070527.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20070602.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20070603.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20070609.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20070610.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20070616.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20070617.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20070623.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20070624.csv'
[Errno

[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20080510.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20080511.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20080517.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20080518.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20080524.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20080525.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20080531.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20080601.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20080607.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20080608.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20080609.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20080614.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20080615.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20080621.csv'
[Errno

[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20090501.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20090502.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20090503.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20090509.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20090510.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20090516.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20090517.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20090523.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20090524.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20090528.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20090529.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20090530.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20090531.csv'
[Errno 2] No such file or directory: '../各交易所期貨持倉量/大商所/m/20090606.csv'
[Errno

In [141]:
contract_id[x]

'm'

In [ ]:
def total_insert(data,table_date,databaseName,code):       
    cursor = db.cursor()
    
    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + "." + code + "_" + table_date+ "(\
    Rank INT(5),\
    Member CHAR(10),\
    Volume FLOAT(5),\
    Increment FLOAT(5),\
    Rank_1 INT(5),\
    Member1 CHAR(10),\
    Buy_Volume FLOAT(5),\
    Increment_1 FLOAT(5),\
    Rank_2 INT(5),\
    Member2 CHAR(10),\
    Sales_Volume FLOAT(5),\
    Increment_2 FLOAT(5) )"

    cursor.execute(sql)
    value=[]

    for i in range(3,len(data)-1):      
        value.append((data.loc[i][0],data.loc[i][1],data.loc[i][2],data.loc[i][3],data.loc[i][4],data.loc[i][5],data.loc[i][6],data.loc[i][7],data.loc[i][8],data.loc[i][9],data.loc[i][10],data.loc[i][11]))

    sql = "INSERT INTO " + databaseName + "." + code + "_" + table_date +" VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"        
    cursor.executemany(sql,value)
    db.commit()
    cursor.close()

# 期貨公司成員輸入

In [ ]:
def create_futures_member(databaseName):
    cursor = db.cursor()
    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".futures_member (\
    Date DATE PRIMARY KEY,\
    Total_volume FLOAT(5),\
    Increment FLOAT(5),\
    Total_buy_volume FLOAT(5),\
    Increment_1 FLOAT(5),\
    Total_sales_volume FLOAT(5),\
    Increment_2 FLOAT(5) ) "

    cursor.execute(sql)

In [ ]:
def futures_member_insert(data,table_date,databaseName):
    cursor = db.cursor()
    
    sql = "INSERT INTO " + databaseName + ".futures_member VALUES( "\
    + table_date +","\
    + data.loc[0][1].replace(',','') +","\
    + data.loc[0][2].replace(',','') +","\
    + data.loc[0][3].replace(',','') +","\
    + data.loc[0][4].replace(',','') +","\
    + data.loc[0][5].replace(',','') +","\
    + data.loc[0][6].replace(',','') + ")"

    cursor.execute(sql)
    db.commit()
    cursor.close()

# 非期貨公司成員輸入

In [ ]:
def create_non_futures_member(databaseName):
    cursor = db.cursor()
    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".none_futures_member (\
    Date DATE PRIMARY KEY,\
    Total_volume FLOAT(5),\
    Increment FLOAT(5),\
    Total_buy_volume FLOAT(5),\
    Increment_1 FLOAT(5),\
    Total_sales_volume FLOAT(5),\
    Increment_2 FLOAT(5) ) "

    cursor.execute(sql)

In [ ]:
def non_futures_member_insert(data,table_date,databaseName):
    cursor = db.cursor()
    sql = "INSERT INTO " + databaseName +".none_futures_member VALUES("\
    + table_date + ","\
    + data.loc[1][1].replace(',','') +","\
    + data.loc[1][2].replace(',','') +","\
    + data.loc[1][3].replace(',','') +","\
    + data.loc[1][4].replace(',','') +","\
    + data.loc[1][5].replace(',','') +","\
    + data.loc[1][6].replace(',','') + ")"

    cursor.execute(sql)
    db.commit()
    cursor.close()

# 總計

In [ ]:
def create_total_volume(databaseName):
    cursor = db.cursor()
    sql = "CREATE TABLE IF NOT EXISTS " + databaseName +".total_volume ( Date DATE PRIMARY KEY, \
    V_Valume FLOAT(5),V_Increment FLOAT(5),\
    B_Valume FLOAT(5),B_Increment FLOAT(5),\
    S_Valume FLOAT(5),S_Increment FLOAT(5))"

    cursor.execute(sql)

In [ ]:
def total_volume_insert(data,table_date,databaseName):
    cursor = db.cursor()
    total_len = len(data)-1
    sql = "INSERT INTO " + databaseName + ".total_volume VALUES("\
    + table_date + ","\
    + data.loc[total_len][2].replace(',','') +","\
    + data.loc[total_len][3].replace(',','') +","\
    + data.loc[total_len][6].replace(',','') +","\
    + data.loc[total_len][7].replace(',','') +","\
    + data.loc[total_len][10].replace(',','') +","\
    + data.loc[total_len][11].replace(',','') + ")"

    cursor.execute(sql)
    db.commit()

# 各公司資訊

## V名次、成交量、增減

In [149]:
def V_info_create(databaseName, company_order):
    cursor = db.cursor()
    name=""
    for i in range(len(company_order)-1):
        name = name + company_order.loc[i][0] + ' TEXT,'
    name = name + company_order.loc[len(company_order)-1][0] + " TEXT"

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName +".v_rank ( Date DATE PRIMARY KEY, " + name + ") ENGINE=MyISAM"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".v_volume ( Date DATE PRIMARY KEY, " + name + ") ENGINE=MyISAM"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".v_increment ( Date DATE PRIMARY KEY, " + name + ") ENGINE=MyISAM"
    cursor.execute(sql)
    cursor.close()

In [131]:
def V_info_insert(data,table_date,databaseName):
    cursor = db.cursor()
    
    empty = ""
    empty = empty + table_date


    sql = "INSERT INTO " + databaseName + ".v_rank (Date) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".v_volume (Date) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".v_increment (Date) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()
    
    for i in range(3,len(data)-1):
        if(data.loc[i][1] == None):
            continue
        
        else:
            sql = "UPDATE " + databaseName + ".v_rank SET " + str(data.loc[i][1]) + "=" + data.loc[i][0] + " WHERE Date=" + table_date
            cursor.execute(sql)
            db.commit()    

    for i in range(3,len(data)-1):
        if(data.loc[i][1] == None):
            continue
            
        else:
            sql = "UPDATE " + databaseName + ".v_volume SET " + str(data.loc[i][1]) + "=" + data.loc[i][2] + " WHERE Date=" + table_date
            cursor.execute(sql)
            db.commit()

    for i in range(3,len(data)-1):
        if(data.loc[i][1] == None):
            continue
            
        else:
            sql = "UPDATE " + databaseName + ".v_increment SET " + data.loc[i][1] + "=" + data.loc[i][3] + " WHERE Date=" + table_date
            cursor.execute(sql)
            db.commit()
    
    cursor.close()

## B名次、買單量、增減

In [156]:
def B_info_create(databaseName, company_order):
    cursor = db.cursor()
    name=""
    for i in range(len(company_order)-1):
        name = name + company_order.loc[i][0] + ' TEXT,'
    name = name + company_order.loc[len(company_order)-1][0] + " TEXT"

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".b_rank ( Date DATE PRIMARY KEY, " + name + ") ENGINE=MyISAM"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".b_oi ( Date DATE PRIMARY KEY, " + name + ") ENGINE=MyISAM"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".b_increment ( Date DATE PRIMARY KEY, " + name + ") ENGINE=MyISAM"
    cursor.execute(sql)
    cursor.close()

In [134]:
def B_info_insert(data,table_date,databaseName):
    cursor = db.cursor()
    name=""

    empty = ""
    empty = empty + table_date 
    
    sql = "INSERT INTO " + databaseName + ".b_rank (Date) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".b_oi (Date) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".b_increment (Date) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()
    
    for i in range(3,len(data)-1):
        if(data.loc[i][5] == None):
            continue  
        else:
            sql = "UPDATE " + databaseName + ".b_rank SET " + str(data.loc[i][5]) + "=" + data.loc[i][4] + " WHERE Date=" + table_date
            cursor.execute(sql)
            db.commit()    

    for i in range(3,len(data)-1):
        if(data.loc[i][5] == None):
            continue
            
        else:
            sql = "UPDATE " + databaseName + ".b_oi SET " + str(data.loc[i][5]) + "=" + data.loc[i][6] + " WHERE Date=" + table_date
            cursor.execute(sql)
            db.commit()

    for i in range(3,len(data)-1):
        if(data.loc[i][5] == None):
            continue
        
        else:
            sql = "UPDATE " + databaseName + ".b_increment SET " + data.loc[i][5] + "=" + data.loc[i][7] + " WHERE Date=" + table_date
            cursor.execute(sql)
            db.commit()

    cursor.close()

## S名次、賣單量、增減

In [157]:
def S_info_create(databaseName, company_order):
    cursor = db.cursor()
    name=""
    for i in range(len(company_order)-1):
        name = name + company_order.loc[i][0] + ' TEXT,'
    name = name + company_order.loc[len(company_order)-1][0] + " TEXT"

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName +".s_rank ( Date DATE PRIMARY KEY, " + name + ") ENGINE=MyISAM"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".s_oi ( Date DATE PRIMARY KEY, " + name + ") ENGINE=MyISAM"
    cursor.execute(sql)

    sql = "CREATE TABLE IF NOT EXISTS " + databaseName + ".s_increment ( Date DATE PRIMARY KEY, " + name + ") ENGINE=MyISAM"
    cursor.execute(sql)
    cursor.close()

In [135]:
def S_info_insert(data,table_date,databaseName):
    cursor = db.cursor()

    empty = ""
    empty = empty + table_date
    
    sql = "INSERT INTO " + databaseName + ".s_rank (Date) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".s_oi (Date) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()

    sql = "INSERT INTO " + databaseName + ".s_increment (Date) VALUES (" + empty + ")"
    cursor.execute(sql)
    db.commit()
    
    for i in range(3,len(data)-1):
        if(data.loc[i][9] == None):
            continue  
        
        else:
            sql = "UPDATE " + databaseName + ".s_rank SET " + str(data.loc[i][9]) + "=" + data.loc[i][8] + " WHERE Date=" + table_date
            cursor.execute(sql)
            db.commit()    

    for i in range(3,len(data)-1):
        if(data.loc[i][9] == None):
            continue
            
        else:
            sql = "UPDATE " + databaseName + ".s_oi SET " + str(data.loc[i][9]) + "=" + data.loc[i][10] + " WHERE Date=" + table_date 
            cursor.execute(sql)
            db.commit()

    for i in range(3,len(data)-1):
        if(data.loc[i][9] == None):
            continue
            
        else:
            sql = "UPDATE " + databaseName +".s_increment SET " + data.loc[i][9] + "=" + data.loc[i][11] + " WHERE Date=" + table_date
            cursor.execute(sql)
            db.commit()

    cursor.close()

# 找出各商品期貨公司

In [103]:
db = MySQLdb.connect("localhost","root","123456","zn_oi_original_data")
db.set_character_set('gbk')

In [104]:
cursor = db.cursor()
sql = 'show tables'
cursor.execute(sql)
data = cursor.fetchall()

In [105]:
company_list = []
for table in data:
    sql = 'select Member,Member1,Member2 from ' + table[0]
    cursor.execute(sql)
    name = cursor.fetchall()   
    for i in name:
        for j in range(3):
            if (i[j] not in company_list and i[j]!=None):
                company_list.append(i[j])

In [106]:
a = pd.DataFrame(company_list)
a.to_excel("../期貨公司名稱/zn_company_name.xlsx",index=None,header=None)

In [107]:
len(company_list)

284

In [14]:
file = open("../cu_company_name", "w")
file.write("\n".join(map(lambda x: str(x), company_list)))
file.close()